# Abstracting Data with DataFrames

## Introduction:

This notebook aims to focus on furthering the understanding of PySpark fundamentals in data structures called DataFrames. Interestingly, DataFrames in PySpark takes advantage of the developments and improvements from Project Tungsten and Catalyst Optimiser.

## Project Tungsten:

The following is a direct quote from https://databricks.com/blog/2015/04/28/project-tungsten-bringing-spark-closer-to-bare-metal.html. It briefly describes the project. 

Project Tungsten will be the largest change to Spark’s execution engine since the project’s inception. It focuses on substantially improving the efficiency of memory and CPU for Spark applications, to push performance closer to the limits of modern hardware. This effort includes three initiatives:

- __Memory Management and Binary Processing__: leveraging application semantics to manage memory explicitly and eliminate the overhead of JVM object model and garbage collection.
- __Cache-aware computation__: algorithms and data structures to exploit memory hierarchy.
- __Code generation__: using code generation to exploit modern compilers and CPUs.

## Catalyst Optimiser:

The following is a direct quote from https://databricks.com/glossary/catalyst-optimizer. It briefly describes the project.

Catalyst is based on functional programming constructs in Scala and designed with these key two purposes:

- Easily add new optimization techniques and features to Spark SQL.
- Enable external developers to extend the optimizer (e.g. adding data source specific rules, support for new data types, etc.).


## Breakdown of this Notebook:
- Creating DataFrames
- Accessing underlying RDDs
- Performance optimisations
- Inferring the schema using Reflections
- Specifying the schema programmatically
- Creating a temporary Table
- Using SQL to interact with DataFrames
- Overview of the DataFrame transformations and Actions.

## 1 PySpark Machine Configuration:

Here it only uses two processing cores from the CPU, and it set up by the following code.

In [1]:
%%configure
{
    "executorCores" : 2
}

In [2]:
from pyspark.sql.types import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
17,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2 Setup the Correct Directory:

In [75]:
import os

# Change the Path:
path = '++++your working directory here++++/Datasets/'
os.chdir(path)
folder_pathway = os.getcwd()

# print(folder_pathway)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3 Creating DataFrames:

First is to create some sample data and column headers as follows.

In [3]:
sample_data = sc.parallelize(
    [(1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02), 
    (2, 'MacBook', 2016, '12"', '8GB', '256GB SSD', 11.04, 7.74, 0.52, 2.03), 
    (3, 'MacBook Air', 2016, '13.3"', '8GB', '128GB SSD', 12.8, 8.94, 0.68, 2.96), 
    (4, 'iMac', 2017, '27"', '64GB', '1TB SSD', 25.6, 8.0, 20.3, 20.8)]
)

col_names = ['Id', 'Model', 'Year', 'ScreenSize', 'RAM', 'HDD', 'W', 'D', 'H', 'Weight']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create the dataFrame from the sample data:

In [4]:
sample_df = spark.createDataFrame(sample_data, col_names)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Inspect:
sample_df.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='15"', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02), Row(Id=2, Model='MacBook', Year=2016, ScreenSize='12"', RAM='8GB', HDD='256GB SSD', W=11.04, D=7.74, H=0.52, Weight=2.03)]

#### As it can be seen, unlike RDDs, a DataFrame is a collection of Row(.......) objects and that a Row(......) object consists of data that is named.

## 3.1 To have a look at the DataFrame Data:

This can be done by using the .show() method.

In [8]:
sample_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+

## 3.2 To have a look at the Schema:

As DataFrames have a schema, and that the columns of the DataFrame have matching datatypes as the original sample_data RDD.

It can be examined like so.

In [9]:
sample_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- HDD: string (nullable = true)
 |-- W: double (nullable = true)
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- Weight: double (nullable = true)

## 3.3 Create DataFrame from a JSON file:

source: https://github.com/kavgan/spark-examples/blob/master/sample-data/description.json

In [23]:
sample_data_json_df = ( spark.read.json(folder_pathway + '/Datasets/' + 'description.json') )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
sample_data_json_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)

In [25]:
sample_data_json_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|                text|      title|
+--------------------+-----------+
|Data (/ˈdeɪtə/ DA...|       Data|
|Big data is a ter...|   Big Data|
|Natural language ...|        NLP|
|Text mining, also...|Text Mining|
+--------------------+-----------+

## 3.4 Create a DataFrame from CSV file:

Unlike loading it in from a JSON file, a CSV file here requires more parameters. These paramters are "header" and "inferSchema". The header parameter will try to assign the right data-type to each column, and the inderSchema parameter will assign strings as the default.

Source: https://github.com/kavgan/spark-examples/blob/master/sample-data/description.csv

In [71]:
sample_data_csv_df = ( spark.read.csv(folder_pathway + '/Datasets/' + 'description.csv', 
                                       header=True, inferSchema=True) )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
sample_data_csv_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)

In [74]:
sample_data_csv_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+
|      title|                text|
+-----------+--------------------+
|       Data|Data (/ˈdeɪtə/ DA...|
|   Big Data|Big data is a ter...|
|        NLP|                text|
|Text Mining|Text mining, also...|
+-----------+--------------------+

## 4 Accessing the underlying RDDs:

